# Exploring GLAD ARD data.

In [1]:
import json
from datetime import datetime

import mercantile
import geopandas as gpd
import pystac
import stac_geoparquet
from lonboard import viz, Map, PolygonLayer
from supermercado import burntiles

In [2]:
# Download from https://glad.umd.edu/ard/home
index_fpath = "/Users/jeffreyalbrecht/Downloads/Global_ARD_tiles/glad_ard_tiles.shp"
df = gpd.read_file(index_fpath)

In [3]:
viz(df)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [4]:
z7_tiles = burntiles.burn(json.loads(df.to_json())['features'], 7)
z7_feats = [mercantile.feature(tile, props={"quadkey": mercantile.quadkey(mercantile.Tile(*tile))}) for tile in z7_tiles]
z7_df = gpd.GeoDataFrame.from_features(z7_feats)

/Users/jeffreyalbrecht/miniconda3/envs/cog-layers/lib/python3.11/site-packages/mercantile/__init__.py:77: FutureWarning: Mercantile 2.0 will require tile x and y to be within the range (0, 2 ** zoom)
  warnings.warn(


In [5]:
# A GeoDataFrame with Polygon geometries
glad_layer = PolygonLayer.from_geopandas(
    df,
    get_fill_color=(189,189,189, 255), get_line_color=[0, 0, 0, 200], line_width_min_pixels=0.25, opacity=0.5
)
z7_layer = PolygonLayer.from_geopandas(
    z7_df,
    get_line_color=[222,45,38, 200],
    line_width_min_pixels=3,
    line_width_max_pixels=2,
    opacity=0.5,
    filled=False
)

m = Map([glad_layer, z7_layer])
m

/Users/jeffreyalbrecht/miniconda3/envs/cog-layers/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:33: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(custom_attribution='', layers=(PolygonLayer(get_fill_color=(189, 189, 189, 255), get_line_color=[0, 0, 0, …

# Create a catalog

In [6]:
with open("glad_ard.jsonl", "w") as outf:
    for row in df.itertuples():
        feat = {
            "id": row.TILE,
            "bbox": row.geometry.bounds,
            "datetime": datetime(2020, 1, 1),
            "geometry": row.geometry.__geo_interface__,
            "properties": {"foo": "bar"},
            "assets": {"img": pystac.Asset(href="gs://123")},
        }
        item = pystac.Item(**feat)
        json.dump(item.to_dict(), outf, separators=(",", ":"))
        outf.write("\n")

In [7]:
stac_geoparquet.arrow.parse_stac_ndjson_to_parquet("glad_ard.jsonl", "glad_ard.parquet")

In [8]:
viz(gpd.read_parquet("glad_ard.parquet"))

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…